In [52]:
import os
import glob
import re
import random
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import warnings
import math
warnings.filterwarnings('ignore')
from google.colab import drive 
drive.mount('/mntDrive')

Drive already mounted at /mntDrive; to attempt to forcibly remount, call drive.mount("/mntDrive", force_remount=True).


In [0]:
eNTERFACE_PATH = "/mntDrive/My Drive/Data/eNTERFACE"
pretrain_path = "/mntDrive/My Drive/Data/r3d50_KMS_200ep.pth"

In [0]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [0]:
class TrainDataset(Dataset):
    def __init__(self, image_paths,transform):
        self.image_paths = image_paths
        self.paths = []
        self.transform = transform
        files = os.listdir(self.image_paths)
        tot_num = 0
        n = 0

        for file in files:
            integers = re.findall(r'\d+', file)
            index = int(integers[0]) 
          
            
            if index < 32:
              self.paths.append(file)
           
        self.paths = sorted_alphanumeric(self.paths)
        #random.shuffle(self.paths)

    def __getitem__(self,index):
        path = self.paths[index]
        dict_data = np.load(self.image_paths+"/"+path)
        data = dict_data['arr_0']
        data = data[0]
        images = []

        outdatasize = 16.0
        datalen = len(data)
        for i in range(int(outdatasize)):
          index = math.floor(datalen/outdatasize*(i))
  
          temp = data[index]
          temp = Image.fromarray(temp, 'RGB')
          
          if self.transform:
                temp = self.transform(temp)
          #display(temp)
          images.append(temp)
        x = np.stack(images)
        #x = torch.from_numpy(data).float().to(device)
      
        if "an" in path:
            y = torch.tensor(0, dtype=torch.long)
        elif "di" in path:
            y = torch.tensor(1, dtype=torch.long)
        elif "fe" in path:
            y = torch.tensor(2, dtype=torch.long)
        elif "ha" in path:
            y = torch.tensor(3, dtype=torch.long)
        elif "sa" in path:
            y = torch.tensor(4, dtype=torch.long)
        elif "su" in path:
            y = torch.tensor(5, dtype=torch.long)
        return x,y
    
    def __len__(self):
        return len(self.paths)

    def l(self):
        return len(self.paths)


In [0]:
class ValDataset(Dataset):
    def __init__(self, image_paths,transform):
        self.image_paths = image_paths
        self.paths = []
        self.transform = transform
        files = os.listdir(self.image_paths)
        tot_num = 0
        n = 0

        for file in files:
            integers = re.findall(r'\d+', file)
            index = int(integers[0]) 
      
            if index > 25:
                self.paths.append(file)
                
        self.paths = sorted_alphanumeric(self.paths)
    def __getitem__(self,index):
        path = self.paths[index]
        dict_data = np.load(self.image_paths+"/"+path)
        data = dict_data['arr_0']
        data = data[0]
        images = []

        outdatasize = 16.0
        datalen = len(data)
        for i in range(int(outdatasize)):
          index = math.floor(datalen/outdatasize*(i))
  
          temp = data[index]
          temp = Image.fromarray(temp, 'RGB')
          if self.transform:
                temp = self.transform(temp)
          images.append(temp)
        x = np.stack(images)
      
        if "an" in path:
            y = torch.tensor(0, dtype=torch.long)
        elif "di" in path:
            y = torch.tensor(1, dtype=torch.long)
        elif "fe" in path:
            y = torch.tensor(2, dtype=torch.long)
        elif "ha" in path:
            y = torch.tensor(3, dtype=torch.long)
        elif "sa" in path:
            y = torch.tensor(4, dtype=torch.long)
        elif "su" in path:
            y = torch.tensor(5, dtype=torch.long)
        return x,y
    
    def __len__(self):
        return len(self.paths)
    def l(self):
        return len(self.paths)

In [0]:
class TestDataset(Dataset):
    def __init__(self, image_paths,transform):
        self.image_paths = image_paths
        self.paths = []
        self.transform = transform
        files = os.listdir(self.image_paths)
        tot_num = 0
        n = 0

        for file in files:
            integers = re.findall(r'\d+', file)
            index = int(integers[0]) 
  
            if index > 32:
              if index < 37:
                self.paths.append(file)
            
        self.paths = sorted_alphanumeric(self.paths)
        #random.shuffle(self.paths)

    def __getitem__(self,index):
        path = self.paths[index]
        dict_data = np.load(self.image_paths+"/"+path)
        data = dict_data['arr_0']
        data = data[0]
        images = []

        outdatasize = 16.0
        datalen = len(data)
        for i in range(int(outdatasize)):
          index = math.floor(datalen/outdatasize*(i))
  
          temp = data[index]
          temp = Image.fromarray(temp, 'RGB')
          if self.transform:
                temp = self.transform(temp)
          images.append(temp)
        x = np.stack(images)
      
        if "an" in path:
            y = torch.tensor(0, dtype=torch.long)
        elif "di" in path:
            y = torch.tensor(1, dtype=torch.long)
        elif "fe" in path:
            y = torch.tensor(2, dtype=torch.long)
        elif "ha" in path:
            y = torch.tensor(3, dtype=torch.long)
        elif "sa" in path:
            y = torch.tensor(4, dtype=torch.long)
        elif "su" in path:
            y = torch.tensor(5, dtype=torch.long)
        return x,y
    
    def __len__(self):
        return len(self.paths)
    def l(self):
        return len(self.paths)

In [0]:
files = os.listdir(eNTERFACE_PATH)

In [59]:
transform = transforms.Compose([
    transforms.Resize((128,171)),
    transforms.CenterCrop(112),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train = TrainDataset(eNTERFACE_PATH,transform)
train_loader = DataLoader(train,batch_size=1, shuffle=True)
val = ValDataset(eNTERFACE_PATH,transform)
val_loader = DataLoader(val,batch_size=1, shuffle=True)
test = TestDataset(eNTERFACE_PATH,transform)
test_loader = DataLoader(test,batch_size=1, shuffle=True)
train_count = train.l()
valid_count = val.l()
test_count = test.l()
print(train_count,valid_count,test_count)
#for data, target in train_loader:
#    print(data.shape,target[0])
    
    #if(target[0] == 0):
    #  a = a + 1
    #if(target[0] == 1):
    #  b = b + 1
    #print(a,b)


803 150 120


In [0]:
max_epoch = 50
batch_size = 1
learning_rate = 0.001

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

!pip install GPUtil
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
import math
from functools import partial

import torch
import torch.nn as nn
import torch.nn.functional as F


def get_inplanes():
    return [64, 128, 256, 512]


def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv1x1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = conv3x3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = conv1x1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 block_inplanes,
                 n_input_channels=3,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 shortcut_type='B',
                 widen_factor=1.0,
                 n_classes=400):
        super().__init__()

        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(n_input_channels,
                               self.in_planes,
                               kernel_size=(conv1_t_size, 7, 7),
                               stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0],
                                       shortcut_type)
        self.layer2 = self._make_layer(block,
                                       block_inplanes[1],
                                       layers[1],
                                       shortcut_type,
                                       stride=2)
        self.layer3 = self._make_layer(block,
                                       block_inplanes[2],
                                       layers[2],
                                       shortcut_type,
                                       stride=2)
        self.layer4 = self._make_layer(block,
                                       block_inplanes[3],
                                       layers[3],
                                       shortcut_type,
                                       stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2),
                                out.size(3), out.size(4))
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(self._downsample_basic_block,
                                     planes=planes * block.expansion,
                                     stride=stride)
            else:
                downsample = nn.Sequential(
                    conv1x1x1(self.in_planes, planes * block.expansion, stride),
                    nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(
            block(in_planes=self.in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def generate_model(model_depth, **kwargs):
    assert model_depth in [10, 18, 34, 50, 101, 152, 200]

    if model_depth == 10:
        model = ResNet(BasicBlock, [1, 1, 1, 1], get_inplanes(), **kwargs)
    elif model_depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], get_inplanes(), **kwargs)
    elif model_depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 50:
        model = ResNet(Bottleneck, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 101:
        model = ResNet(Bottleneck, [3, 4, 23, 3], get_inplanes(), **kwargs)
    elif model_depth == 152:
        model = ResNet(Bottleneck, [3, 8, 36, 3], get_inplanes(), **kwargs)
    elif model_depth == 200:
        model = ResNet(Bottleneck, [3, 24, 36, 3], get_inplanes(), **kwargs)

    return model

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as weight_init
class ConvNet(nn.Module):
    
    def __init__(self,resnet): 
    # you can add any additional parameters you want 
    # TODO:
    # You should create your neural network here
        super(ConvNet,self).__init__()
      # convolutional layer
        self.resnet = resnet
        
      # fully connected layers
        self.fc1 = nn.Linear(512, 6)
        self.relu = nn.ReLU(inplace=True)
        self.softmax = nn.LogSoftmax()
    
          
    def forward(self, X): 
    # you can add any additional parameters you want
    # TODO:
    # Forward propagation implementation should be here
        #X = X[None,:,:,:]
        #print(X.shape)
        X = self.resnet(X)
        X = self.fc1(X)
        return X

In [0]:
def train_model(criterion, optimizer, model):
    train_loss = 0.0
    train_acc = 0.0
    
    batch_acc = []
    batch_loss = []
    num = 0
    # training the model with the train dataset.
    for train_images, train_labels in train_loader:
        train_images = train_images.to( device ,dtype=torch.float)
        train_labels = train_labels.to( device ,dtype=torch.float) 

      
        train_labels_head = model.forward(train_images.permute(0,2,1,3,4))

        loss = criterion(train_labels_head, train_labels.long())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  

        predictions = torch.max( train_labels_head , 1)[1] 
      
        acurracy = torch.sum( predictions == train_labels )

        accur_save = float(acurracy.float())/1
        loss_save= float(loss.item())/1

        train_acc += acurracy 
        train_loss += loss.item()
        num = num + 1
        if num % 1000 == 0:
          print(num,train_acc/num)

        train_acc_print = accur_save
        batch_acc.append(train_acc_print)
        train_loss_print = loss_save
        batch_loss.append(train_loss_print)

    return train_loss, train_acc,batch_acc,batch_loss

In [0]:
def validate_model(criterion, model):
    valid_loss = 0.0
    valid_acc  = 0.0
    valid_labels_tot = []
    pred = []
    # testing the current model within the current epoch.
    for valid_images, valid_labels in val_loader:
        #GPUs = GPU.getGPUs()
        # XXX: only one GPU on Colab and isn’t guaranteed
        #gpu = GPUs[0]

        #process = psutil.Process(os.getpid())
        acurracy,loss,predictions = step(valid_images,valid_labels)
        # calculating acurracy for the cur-
        # rent batch for the test dataset.
        
    
        # summing test acurracy and 
        # loss wit the remaining batches.
        valid_acc  += acurracy 
        valid_loss += loss.item()
        valid_labels_tot.append(valid_labels)
        pred.append(predictions)
    return valid_loss, valid_acc,pred,valid_labels_tot
def step(valid_images,valid_labels):
  # registering test images and labels 
  # to device to benefit from GPU
  valid_images = valid_images.to( device ,dtype=torch.float)
  valid_labels = valid_labels.to( device ,dtype=torch.float)
  #valid_frame = valid_frame.to( device ,dtype=torch.float)
  optimizer.zero_grad()
  if valid_images.shape[1] > 120:
    temp = int((valid_images.shape[1] - 120) / 2) + 1
    tempAr = valid_images[0][temp:-temp]
    #print(tempAr.shape)
    valid_images = torch.zeros([1,tempAr.shape[0],3,224,224], dtype=torch.float).to( device ,dtype=torch.float)
    valid_images[0] = tempAr
  # calculating loss for test 
  # dataset for the current batch.
  #if(valid_frame[0][0].to(dtype=torch.int32) != 0):
  valid_labels_head = model.forward(valid_images.permute(0,2,1,3,4))
  loss = criterion(valid_labels_head, valid_labels.long())
  predictions = torch.max( valid_labels_head, 1)[1] 
  acurracy = torch.sum( predictions == valid_labels )
  
  return acurracy,loss,predictions

In [0]:
def test_model(criterion, model):
    test_loss = 0.0
    test_acc  = 0.0
    test_labels_tot = []
    pred = []
    i = 1
    # testing the current model within the current epoch.
    for test_images, test_labels in val_loader:
        
        # registering test images and labels 
        # to device to benefit from GPU
        test_images = test_images.to( device ,dtype=torch.float)
        test_labels = test_labels.to( device ,dtype=torch.float)
        #test_frame = test_frame.to( device ,dtype=torch.float)
        if test_images.shape[1] > 120:
          temp = int((test_images.shape[1] - 120) / 2) + 1
          tempAr = test_images[0][temp:-temp]
          #print(tempAr.shape)
          test_images = torch.zeros([1,tempAr.shape[0],3,224,224], dtype=torch.float).to( device ,dtype=torch.float)
          test_images[0] = tempAr
        # calculating loss for test 
        # dataset for the current batch.
        test_labels_head = model.forward(test_images.permute(0,2,1,3,4))
        loss = criterion(test_labels_head, test_labels.long())
        #print(torch.max( test_labels_head, 1)[1] ,test_labels.long())
        # calculating acurracy for the cur-
        # rent batch for the test dataset.
        predictions = torch.max( test_labels_head, 1)[1] 
        acurracy = torch.sum( predictions == test_labels )
        i = i + 1
        # summing test acurracy and 
        # loss wit the remaining batches.
        test_acc  += acurracy 
        print(test_acc/i)
        test_loss += loss.item()
        test_labels_tot.append(test_labels)
        pred.append(predictions)
    return test_loss, test_acc, pred, test_labels_tot

In [0]:
import torch.nn.init as weight_init

resnet = generate_model(50,n_classes=1139)
print('loading pretrained model {}'.format(pretrain_path))
pretrain = torch.load(pretrain_path, map_location='cpu')
resnet.load_state_dict(pretrain['state_dict'])

i= 0
for name,child in resnet.named_children(): 
    for name2,param in child.named_parameters():
      if i < 20:
        print(name2,i)
        i = i + 1
        param.requires_grad = False

resnet.relu = nn.ReLU(inplace=True)
resnet.fc = nn.Linear(2048, 512)
resnet.relu2 = nn.ReLU(inplace=True)
model = ConvNet(resnet).to(device)
#model.load_state_dict(torch.load('/mntDrive/My Drive/Colab Notebooks/video_p.pth'))

#model = torch.load('/mntDrive/My Drive/Colab Notebooks/cont3.pth').to(device)

print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate) # you can play with momentum and weight_decay parameters as well

train_losses = []
valid_losses = []
train_accurracies = []
valid_accurracies = []
batch_acc_tot  = []
batch_loss_tot  = []
best_acc = 0

for epoch in range(max_epoch):
    
    train_loss = 0.0
    valid_loss = 0.0
    train_acc = 0.0
    valid_acc = 0.0
  
    # training the model with the train dataset.
    train_loss, train_acc,batch_acc,batch_loss = train_model(criterion, optimizer, model)
    batch_acc_tot.append(batch_acc)
    batch_loss_tot.append(batch_loss)
    # validating the current model within the current epoch.
    with torch.no_grad():
      valid_loss, valid_acc, predictions, valid_labels = validate_model(criterion, model)

    valid_acc = float(valid_acc.float()) / valid_count
    valid_accurracies.append( valid_acc )

    pred = np.array([])
    for i in predictions:
      pred = np.concatenate((pred,i.cpu().detach().numpy()),axis=None)

    labels = np.array([])
    for i in valid_labels:
      labels = np.concatenate((labels,i.cpu().detach().numpy()),axis=None)

    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    import matplotlib.pyplot as plt
    cm = confusion_matrix(labels, pred)
    cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis].astype('float')
    
    f, ax = plt.subplots(figsize = (10, 10))
    sns.heatmap(cmn, annot = True, linewidths=1, linecolor='black', fmt='.2f', ax=ax)
    plt.xlabel("Test Predictions")
    plt.ylabel("Test Labels")
    plt.show()
  

    print("EPOCH --> " + str(epoch + 1))
    print("----------------------------------------------") 
    train_acc = float(train_acc.float()) / train_count
    train_accurracies.append( train_acc )
    print("Train Acurracy: " + str( float(train_acc) ))
    
    train_loss = train_loss / train_count
    train_losses.append( train_loss )
    print("Train Loss: " + str(train_loss) + "\n")

    
    print("Validation Acurracy: " + str( float(valid_acc) ))    
    
    valid_loss = valid_loss / valid_count
    valid_losses.append( valid_loss )
    print("Validation Loss: " + str(valid_loss))
    print("----------------------------------------------\n")